In [591]:
from cot import Collection
import pandas as pd
import numpy as np
import krippendorff
import random
import scipy
from scipy import stats
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = "{:.2f}".format
%precision 2

'%.2f'

In [592]:
coll = Collection.from_json("/home/kon/work/ThoughtSource/libs/cot/cot/datasets/thoughtsource/thoughtsource_33_paper.json")

In [593]:
ev = coll.evaluate(overwrite=True)

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

In [594]:
df = coll.collection_to_dataframe()

In [595]:
replacements = {
    'None_None': 'Direct',
    'qa-10_None': 'Ins-10',
    'qa-12_None': 'Ins-12',
    'qa-13_None': 'Ins-13',
    'qa-16_None': 'Ins-16',
    'qa-17_None': 'Ins-17',
    'zhou-01-ins_None': 'Zhou-Ins',
    'refl-01_None': 'Reflection',
    'None_kojima-01': 'Kojima',
    'None_zhou-01': 'Zhou'
}

In [596]:
df['prompt'].replace(replacements, inplace=True)

In [597]:
df.head(1)

,dataset,split,id,type,answer_label,prompt,instruction,cot_trigger,answer_from_choices,correct_answer,model
0,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,multiplechoice,E,Direct,None,None,E,True,command-xlarge-nightly


In [598]:
acc_model_prompt = df.groupby(["model", "prompt"]).mean("correct_answer").pivot_table(index="model", columns="prompt", values="correct_answer")
acc_model_prompt

prompt,Direct,Ins-10,Ins-12,Ins-13,Ins-16,Ins-17,Kojima,Reflection,Zhou,Zhou-Ins
model,,,,,,,,,,
command-xlarge-nightly,0.45,0.44,0.46,0.48,0.52,0.47,0.42,0.43,0.51,0.47
flan-T5-xxl,0.62,0.60,0.63,0.61,0.54,0.61,0.62,0.57,0.58,0.59
gpt-3.5-turbo,0.71,0.67,0.67,0.66,0.65,0.68,0.69,0.65,0.70,0.67
gpt-4,0.79,0.83,0.80,0.82,0.80,0.78,0.83,0.80,0.86,0.81
text-davinci-002,0.57,0.53,0.58,0.57,0.61,0.49,0.53,0.54,0.61,0.53
text-davinci-003,0.60,0.61,0.64,0.60,0.56,0.59,0.58,0.61,0.63,0.63


In [599]:
kripp_model_prompt.pivot_table(index="model", columns="prompt", values="mean")

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
model,,,,,,,,,,
command-xlarge-nightly,0.23,0.21,0.31,NaN,NaN,NaN,0.34,0.28,NaN,NaN
flan-T5-xxl,0.46,0.46,0.39,0.43,0.47,0.45,0.40,0.45,0.36,0.40
gpt-3.5-turbo,0.62,0.66,0.63,0.62,0.60,0.61,0.61,0.59,0.60,0.64
gpt-4,0.72,0.81,0.84,0.78,0.79,0.77,0.77,0.74,0.77,0.79
text-davinci-002,0.40,0.39,0.53,0.30,0.42,0.46,0.52,0.36,0.35,0.30
text-davinci-003,0.43,0.44,0.50,0.45,0.50,0.47,0.40,0.44,0.48,0.48


In [600]:
acc_dataset_model = df.groupby(["model", "dataset"]).mean("correct_answer").pivot_table(index="dataset", columns="model", values="correct_answer")
acc_dataset_model

model,command-xlarge-nightly,flan-T5-xxl,gpt-3.5-turbo,gpt-4,text-davinci-002,text-davinci-003
dataset,,,,,,
commonsense_qa,0.66,0.83,0.68,0.75,0.69,0.69
med_qa,0.26,0.21,0.50,0.64,0.27,0.34
medmc_qa,0.31,0.34,0.59,0.78,0.40,0.39
open_book_qa,0.57,0.77,0.76,0.91,0.56,0.69
strategy_qa,0.25,0.62,0.60,0.82,0.53,0.62
worldtree,0.75,0.81,0.91,0.98,0.88,0.88


In [601]:
acc_dataset_prompt = df.groupby(["prompt", "dataset"]).mean("correct_answer").pivot_table(index="dataset", columns="prompt", values="correct_answer")
acc_dataset_prompt

prompt,Direct,Ins-10,Ins-12,Ins-13,Ins-16,Ins-17,Kojima,Reflection,Zhou,Zhou-Ins
dataset,,,,,,,,,,
commonsense_qa,0.72,0.75,0.72,0.74,0.66,0.65,0.70,0.76,0.72,0.76
med_qa,0.37,0.36,0.38,0.37,0.40,0.36,0.37,0.35,0.39,0.36
medmc_qa,0.45,0.47,0.49,0.47,0.51,0.48,0.42,0.42,0.53,0.43
open_book_qa,0.73,0.72,0.74,0.66,0.66,0.70,0.68,0.72,0.78,0.72
strategy_qa,0.60,0.52,0.52,0.61,0.61,0.59,0.67,0.49,0.60,0.53
worldtree,0.87,0.85,0.91,0.89,0.83,0.84,0.83,0.86,0.87,0.90


In [602]:
# KRIPPENDORFF ALPHA all datasets together, pooling over datasets
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (model, prompt), group in df.groupby(['model', 'prompt']):
    unique_datasets = group['dataset'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):
        random_dataset = random.choice(unique_datasets)
        random_group = group[group['dataset'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        # since we only sample from few samples here, sometimes there are only samples with no label sampled
        try:
            metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')
            dataset_means[random_dataset].append(metric)

        except Exception as e:
                print(e)
                print(random_dataset, model, prompt)
                continue
        

    all_means = [np.mean(means) for means in dataset_means.values()]
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'model': model,
        'prompt': prompt,
        'mean': mean,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

/home/kon/work/ThoughtSource/venv/lib/python3.10/site-packages/krippendorff/krippendorff.py:364: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - (o * d).sum() / (e * d).sum()


There has to be at least one unit with values assigned by at least two coders.
strategy_qa command-xlarge-nightly Ins-12
There has to be at least one unit with values assigned by at least two coders.
strategy_qa command-xlarge-nightly Ins-13
There has to be at least one unit with values assigned by at least two coders.
strategy_qa command-xlarge-nightly Ins-13
There has to be at least one unit with values assigned by at least two coders.
strategy_qa command-xlarge-nightly Zhou-Ins


In [603]:
[len(x) for x in dataset_means.values()]

[173, 168, 164, 156, 174, 165]

In [604]:
kripp_model_prompt = results_df.copy()
kripp_model_prompt.head(1)

,model,prompt,mean,ci_lower,ci_upper
0,command-xlarge-nightly,Direct,0.23,0.13,0.33


In [605]:
kripp_model_prompt_mean = kripp_model_prompt.pivot_table(values='mean', index='model', columns='prompt')
kripp_model_prompt_ci_lower = kripp_model_prompt.pivot_table(values='ci_lower', index='model', columns='prompt')
kripp_model_prompt_ci_upper = kripp_model_prompt.pivot_table(values='ci_upper', index='model', columns='prompt')
kripp_model_prompt_mean

prompt,Direct,Ins-10,Ins-12,Ins-13,Ins-16,Ins-17,Kojima,Reflection,Zhou,Zhou-Ins
model,,,,,,,,,,
command-xlarge-nightly,0.23,NaN,NaN,NaN,0.34,0.28,0.20,NaN,0.31,NaN
flan-T5-xxl,0.46,0.43,0.48,0.45,0.40,0.45,0.46,0.36,0.39,0.41
gpt-3.5-turbo,0.62,0.63,0.60,0.61,0.61,0.59,0.66,0.58,0.63,0.64
gpt-4,0.72,0.78,0.79,0.77,0.77,0.74,0.81,0.76,0.84,0.80
text-davinci-002,0.40,0.31,0.43,0.45,0.52,0.36,0.40,0.35,0.52,0.31
text-davinci-003,0.43,0.44,0.49,0.47,0.40,0.44,0.45,0.48,0.50,0.48


In [606]:
# acc_model_prompt/kripp_model_prompt.pivot_table(index="model", columns="prompt", values="mean")

In [607]:
# KRIPPENDORFF ALPHA dataset by prompt sum over models

# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (dataset, prompt), group in df.groupby(['dataset', 'prompt']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')

        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        # 'model': model,
        'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)

In [608]:
kripp_dataset_prompt = results_df.copy()
kripp_dataset_prompt.head(1)

,dataset,prompt,mean,ci_lower,ci_upper
0,commonsense_qa,Direct,0.68,0.60,0.76


In [609]:
kripp_dataset_prompt_mean = kripp_dataset_prompt.pivot_table(values='mean', index='dataset', columns='prompt')
kripp_dataset_prompt_ci_lower = kripp_dataset_prompt.pivot_table(values='ci_lower', index='dataset', columns='prompt')
kripp_dataset_prompt_ci_upper = kripp_dataset_prompt.pivot_table(values='ci_upper', index='dataset', columns='prompt')
kripp_dataset_prompt_mean

prompt,Direct,Ins-10,Ins-12,Ins-13,Ins-16,Ins-17,Kojima,Reflection,Zhou,Zhou-Ins
dataset,,,,,,,,,,
commonsense_qa,0.68,0.73,0.73,0.75,0.68,0.63,0.69,0.73,0.72,0.73
med_qa,0.21,0.20,0.23,0.21,0.26,0.21,0.23,0.19,0.24,0.20
medmc_qa,0.28,0.30,0.35,0.31,0.36,0.32,0.25,0.25,0.37,0.27
open_book_qa,0.65,0.65,0.67,0.60,0.64,0.63,0.61,0.66,0.74,0.65
strategy_qa,0.24,0.29,0.32,0.46,0.40,0.33,0.44,0.21,0.38,0.31
worldtree,0.84,0.82,0.88,0.87,0.82,0.78,0.79,0.82,0.83,0.86


In [610]:
# kripp_dataset_prompt_mean.std(1)

In [611]:
# kripp_dataset_prompt_mean.std(0)

In [612]:
# acc_dataset_prompt/kripp_dataset_prompt.pivot_table(values='mean', index='dataset', columns='prompt')

In [613]:
# KRIPPENDORFF ALPHA dataset by model, sum over prompts
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (dataset, model), group in df.groupby(['dataset', 'model']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')

        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        'model': model,
        # 'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)

In [614]:
kripp_dataset_model = results_df.copy()
kripp_dataset_model.head(1)

,dataset,model,mean,ci_lower,ci_upper
0,commonsense_qa,command-xlarge-nightly,0.57,0.50,0.64


In [615]:
kripp_dataset_model_mean = kripp_dataset_model.pivot_table(values='mean', index='dataset', columns='model')
kripp_dataset_model_ci_lower = kripp_dataset_model.pivot_table(values='ci_lower', index='dataset', columns='model')
kripp_dataset_model_ci_upper = kripp_dataset_model.pivot_table(values='ci_upper', index='dataset', columns='model')
kripp_dataset_model_mean

model,command-xlarge-nightly,flan-T5-xxl,gpt-3.5-turbo,gpt-4,text-davinci-002,text-davinci-003
dataset,,,,,,
commonsense_qa,0.57,0.81,0.70,0.82,0.68,0.68
med_qa,0.06,0.01,0.41,0.57,0.09,0.18
medmc_qa,0.08,0.09,0.51,0.73,0.20,0.21
open_book_qa,0.43,0.69,0.77,0.91,0.45,0.66
strategy_qa,0.05,0.24,0.44,0.69,0.22,0.22
worldtree,0.67,0.77,0.89,0.97,0.84,0.84


In [616]:
# acc_dataset_model/kripp_dataset_model.pivot_table(values='mean', index='dataset', columns='model')

In [617]:
# KRIPPENDORFF ALPHA only DATASET, sum over prompts and models
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for dataset, group in df.groupby('dataset'):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')

        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        # 'model': model,
        # 'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)

In [618]:
kripp_dataset = results_df.copy()

In [619]:
kripp_dataset

,dataset,mean,ci_lower,ci_upper
0,commonsense_qa,0.71,0.68,0.73
1,med_qa,0.22,0.19,0.25
2,medmc_qa,0.31,0.27,0.34
3,open_book_qa,0.65,0.63,0.68
4,strategy_qa,0.34,0.30,0.39
5,worldtree,0.83,0.81,0.85


In [620]:
# KRIPPENDORFF ALPHA model scores by summing over prompts and datasets
# BOOTSTRAPPING SEPERATELY FOR EACH DATASET, SINCE THE NUMBER OF CHOICES IS DIFFERENT
import numpy as np
import pandas as pd
import random
import scipy.stats

n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for model, group in df.groupby('model'):
    unique_datasets = group['dataset'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):

        random_dataset = random.choice(unique_datasets)
        random_group = group[group['dataset'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')

        dataset_means[random_dataset].append(metric)

    all_means = [np.mean(means) for means in dataset_means.values()]
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'model': model,
        'mean': mean,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [621]:
kripp_model = results_df.copy()

In [622]:
kripp_model

,model,mean,ci_lower,ci_upper
0,command-xlarge-nightly,0.31,0.27,0.35
1,flan-T5-xxl,0.44,0.41,0.46
2,gpt-3.5-turbo,0.62,0.59,0.65
3,gpt-4,0.78,0.76,0.81
4,text-davinci-002,0.41,0.38,0.44
5,text-davinci-003,0.46,0.43,0.49


In [623]:
# KRIPPENDORFF ALPHA prompt scores by summing over models and datasets
# BOOTSTRAPPING SEPERATELY FOR EACH DATASET, SINCE THE NUMBER OF CHOICES IS DIFFERENT
import numpy as np
import pandas as pd
import random
import scipy.stats

n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for prompt, group in df.groupby('prompt'):
    unique_datasets = group['dataset'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):

        random_dataset = random.choice(unique_datasets)
        random_group = group[group['dataset'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')

        dataset_means[random_dataset].append(metric)

    all_means = [np.mean(means) for means in dataset_means.values()]
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'prompt': prompt,
        'mean': mean,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [624]:
kripp_prompt = results_df.copy()

In [625]:
kripp_prompt

,prompt,mean,ci_lower,ci_upper
0,Direct,0.48,0.44,0.52
1,Ins-10,0.50,0.46,0.54
2,Ins-12,0.53,0.49,0.57
3,Ins-13,0.53,0.49,0.57
4,Ins-16,0.53,0.49,0.57
5,Ins-17,0.48,0.44,0.52
6,Kojima,0.50,0.47,0.54
7,Reflection,0.47,0.44,0.51
8,Zhou,0.55,0.51,0.58
9,Zhou-Ins,0.50,0.47,0.54


In [638]:
# df = kripp_prompt.copy()
# df = kripp_model.copy()
df = kripp_dataset.copy()

In [639]:
df['mean_ci'] = df['mean'].round(2).astype(str) + " (" + df['ci_lower'].round(2).astype(str) + ", " + df['ci_upper'].round(2).astype(str) + ")"

# Drop the original 'mean', 'ci_lower', and 'ci_upper' columns
df = df.drop(columns=['mean', 'ci_lower', 'ci_upper'])

# Convert the DataFrame to a LaTeX table
latex_table = df.to_latex(column_format="l" + "c" * len(df.columns), float_format="{:.2f}".format, escape=False, index=False)

print(latex_table)

\begin{tabular}{lcc}
\toprule
       dataset &           mean_ci \\
\midrule
commonsense_qa & 0.71 (0.68, 0.73) \\
        med_qa & 0.22 (0.19, 0.25) \\
      medmc_qa & 0.31 (0.27, 0.34) \\
  open_book_qa & 0.65 (0.63, 0.68) \\
   strategy_qa &  0.34 (0.3, 0.39) \\
     worldtree & 0.83 (0.81, 0.85) \\
\bottomrule
\end{tabular}



/tmp/ipykernel_273/3894489030.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(column_format="l" + "c" * len(df.columns), float_format="{:.2f}".format, escape=False, index=False)


In [631]:
mean_pivot = kripp_dataset_model_mean.copy().round(2)
ci_lower = kripp_dataset_model_ci_lower.copy().round(2)
ci_upper = kripp_dataset_model_ci_upper.copy().round(2)

In [632]:
# Combine the DataFrames

final_df = mean_pivot.copy()
for col in mean_pivot.columns:
    final_df[col] = mean_pivot[col].round(4).astype(str) + " ({\\scriptsize " + ci_lower[col].round(4).astype(str) + ", " + ci_upper[col].round(4).astype(str) + "})"

# Apply custom formatting (use LaTeX format for a scientific paper)
latex_table = final_df.to_latex(column_format="l" + "c" * len(final_df.columns), float_format="{:.2f}".format, escape=False)

print(latex_table)


\begin{tabular}{lcccccc}
\toprule
model &            command-xlarge-nightly &                       flan-T5-xxl &                    gpt-3.5-turbo &                            gpt-4 &                 text-davinci-002 &                 text-davinci-003 \\
dataset        &                                   &                                   &                                  &                                  &                                  &                                  \\
\midrule
commonsense_qa &    0.57 ({\scriptsize 0.5, 0.64}) &   0.81 ({\scriptsize 0.75, 0.85}) &   0.7 ({\scriptsize 0.64, 0.76}) &  0.82 ({\scriptsize 0.76, 0.87}) &  0.68 ({\scriptsize 0.62, 0.74}) &  0.68 ({\scriptsize 0.62, 0.74}) \\
med_qa         &   0.06 ({\scriptsize 0.01, 0.13}) &  0.01 ({\scriptsize -0.04, 0.07}) &  0.41 ({\scriptsize 0.35, 0.48}) &   0.57 ({\scriptsize 0.5, 0.64}) &  0.09 ({\scriptsize 0.03, 0.15}) &  0.18 ({\scriptsize 0.12, 0.25}) \\
medmc_qa       &   0.08 ({\scriptsize 0.01, 0.

/tmp/ipykernel_273/1925743105.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = final_df.to_latex(column_format="l" + "c" * len(final_df.columns), float_format="{:.2f}".format, escape=False)


### test runs for checking scores

In [449]:
# test run to see if the pooling is correct for the dataset option
# bootstrapping per dataset is not needed for the dataset option, we can check if it leads to the same results

import numpy as np
import pandas as pd
import random
import scipy.stats

n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (dataset), group in df.groupby(['dataset']):
    unique_datasets = group['model'].unique()
    dataset_means = {dataset: [] for dataset in unique_datasets}

    for _ in range(n_bootstraps):

        random_dataset = random.choice(unique_datasets)
        random_group = group[group['model'] == random_dataset]

        answer_label = random_group['answer_label'].sample(n=len(random_group), replace=True)
        indices = answer_label.index
        answer_from_choices = random_group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)], level_of_measurement='nominal')

        dataset_means[random_dataset].append(metric)

    all_means = [np.mean(means) for means in dataset_means.values()]
    mean = np.mean(all_means)

    # Calculate the pooled standard deviation
    dataset_stds = {dataset: np.std(means) for dataset, means in dataset_means.items()}

    squared_stds = [std ** 2 for std in dataset_stds.values()]
    pooled_std_dev = np.sqrt(sum(squared_stds) / len(squared_stds))

    # Calculate the margin of error using the z-score
    z_score = scipy.stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z_score * (pooled_std_dev / np.sqrt(len(unique_datasets)))

    # Compute the confidence interval
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error

    results.append({
        'dataset': dataset,
        'mean': mean,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

/tmp/ipykernel_273/1604803771.py:16: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (dataset), group in df.groupby(['dataset']):


In [450]:
# test if the pooling is correct for the dataset option
results_df

,dataset,mean,ci_lower,ci_upper
0,commonsense_qa,0.7111,0.6876,0.7346
1,med_qa,0.2207,0.1956,0.2459
2,medmc_qa,0.3022,0.2737,0.3306
3,open_book_qa,0.6512,0.6248,0.6777
4,strategy_qa,0.3085,0.2605,0.3565
5,worldtree,0.8329,0.8139,0.8519


In [451]:
# should be very similar to
kripp_dataset

,dataset,mean,ci_lower,ci_upper
0,commonsense_qa,0.7073,0.6819,0.7314
1,med_qa,0.2184,0.1920,0.2456
2,medmc_qa,0.3051,0.2749,0.3358
3,open_book_qa,0.6516,0.6258,0.6780
4,strategy_qa,0.3436,0.2961,0.3935
5,worldtree,0.8325,0.8125,0.8525


these two should roughly be the same

In [461]:
kripp_dataset_model.pivot_table(values='mean', index='dataset', columns='model').mean()

model
command-xlarge-nightly   0.3103
flan-T5-xxl              0.4359
gpt-3.5-turbo            0.6212
gpt-4                    0.7827
text-davinci-002         0.4138
text-davinci-003         0.4648
dtype: float64

In [462]:
kripp_model[['model','mean']]

,model,mean
0,command-xlarge-nightly,0.3135
1,flan-T5-xxl,0.4325
2,gpt-3.5-turbo,0.6334
3,gpt-4,0.7835
4,text-davinci-002,0.4185
5,text-davinci-003,0.4698


In [ ]:
# # INCORRECT SINCE SUMMING OVER DATASETS
# # RESULTS WILL BE TOO HIGH

# # KRIPPENDORFF ALPHA all datasets together
# # Define bootstrapping parameters
# n_bootstraps = 1000
# confidence_level = 0.95
# np.random.seed(42)  # Set the seed for reproducibility

# # Perform bootstrapping and calculate means and confidence intervals
# results = []

# for (model, prompt), group in df.groupby(['model', 'prompt']):
#     bootstrapped_means = []
#     for _ in range(n_bootstraps):
#         answer_label = group['answer_label'].sample(n=len(group), replace=True)
#         indices = answer_label.index
#         answer_from_choices = group.loc[indices, 'answer_from_choices']

#         metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')

#         bootstrapped_means.append(metric)
    
#     mean = np.mean(bootstrapped_means)
#     lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
#     upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
#     results.append({
#         # 'dataset': dataset,
#         'model': model,
#         'prompt': prompt,
#         'mean': mean,
#         'ci_lower': lower,
#         'ci_upper': upper
#     })

# # Convert the results to a DataFrame and pivot
# results_df = pd.DataFrame(results)

# results_df.pivot_table(values='mean', index='model', columns='prompt')

In [ ]:
# # Define bootstrapping parameters
# n_bootstraps = 1000
# confidence_level = 0.95
# np.random.seed(42)  # Set the seed for reproducibility

# # Perform bootstrapping and calculate means and confidence intervals
# results = []

# for (model, prompt, dataset), group in df.groupby(['model', 'prompt', 'dataset']):
#     bootstrapped_means = []
#     for _ in range(n_bootstraps):
#         resampled = group['correct_answer'].sample(n=len(group), replace=True)
#         bootstrapped_means.append(resampled.mean())
    
#     mean = np.mean(bootstrapped_means)
#     lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
#     upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
#     results.append({
#         'dataset': dataset,
#         'model': model,
#         'prompt': prompt,
#         'mean': mean,
#         'ci_lower': lower,
#         'ci_upper': upper
#     })

# # Convert the results to a DataFrame and pivot
# results_df = pd.DataFrame(results)
# mean_pivot = results_df.pivot_table(values='mean', index='model', columns='prompt')
# ci_pivot_lower = results_df.pivot_table(values='ci_lower', index='model', columns='prompt')
# ci_pivot_upper = results_df.pivot_table(values='ci_upper', index='model', columns='prompt')

In [ ]:
# # ACCURACY BOOTSRTAPPING
# # Define bootstrapping parameters
# n_bootstraps = 1000
# confidence_level = 0.95
# np.random.seed(42)  # Set the seed for reproducibility

# # Perform bootstrapping and calculate means and confidence intervals
# results = []

# for (model, prompt), group in df.groupby(['model', 'prompt']):
#     bootstrapped_means = []
#     for _ in range(n_bootstraps):
#         resampled = group['correct_answer'].sample(n=len(group), replace=True)
#         bootstrapped_means.append(resampled.mean())
    
#     mean = np.mean(bootstrapped_means)
#     lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
#     upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
#     results.append({
#         'model': model,
#         'prompt': prompt,
#         'mean': mean,
#         'ci_lower': lower,
#         'ci_upper': upper
#     })

# # Convert the results to a DataFrame and pivot
# results_df = pd.DataFrame(results)

### inspection of the dataframe

In [481]:
filtered_df = df[(df.dataset == "strategy_qa")& (df.model == "command-xlarge-nightly") & (df.prompt == "None_zhou-01")]
# filtered_df.head(1)

In [ ]:
# filtered_df[['answer', 'answer_label']]

In [ ]:
# filtered_df["correct_answer"].unique(), filtered_df["answer_from_choices"].unique()

In [ ]:
# filtered_df["correct_answer"].value_counts(), filtered_df["answer_from_choices"].value_counts()

In [ ]:
# filtered_df["correct_answer"].count(), filtered_df["answer_from_choices"].count()

In [ ]:
krippendorff.alpha(reliability_data=[list(filtered_df["answer_label"]), list(filtered_df["answer_from_choices"].replace("", np.nan))],level_of_measurement='nominal')

0.0493

In [ ]:
# filtered_df[["answer","answer_from_choices"]][:200]

In [ ]:
# df.groupby(['dataset'])['number_choices'].unique()

### functions extra, non pandas

In [29]:
def table_function_new(eval:dict):
    import pandas as pd
    eval_dict = pd.json_normalize(eval).to_dict('records')[0]
    eval_list = list(eval_dict.keys())
    datasets = sorted(list(eval.keys()))

    models = []
    prompts = []
    for i in eval_list:
        # fast fix for chat gpt model:
        i = i.replace("gpt-3.5-turbo","gpt-3-5-turbo")
        dataset,split,metric,model,prompt = i.split(".")
        model = model.replace("gpt-3-5-turbo","gpt-3.5-turbo")
        if model not in models:
            models.append(model)
        if prompt not in prompts:
            prompts.append(prompt)
            
    models = sorted(models)

    if "None_None_None" in prompts: prompts.remove("None_None_None")

    # no instructions implemented yet
    # instructions = []
    cot_triggers = []
    for i in prompts:
        instruction, cot_trigger, _ = i.split("_")
        # if instruction not in instructions:
        #     instructions.append(instruction)

        # old: only cot_trigger
        # if cot_trigger not in cot_triggers:
        #     cot_triggers.append(cot_trigger)

        # new: instruction + cot_trigger
        if cot_trigger not in cot_triggers:
            cot_triggers.append(instruction + "_" + cot_trigger)

    cot_triggers = sorted(cot_triggers)

    cot_trigger_header = sorted(cot_triggers*len(models))
    model_header = models*len(cot_triggers)

    # Create a dictionary to store the data
    data_dict = {}
    for k,v in eval_dict.items():
        # fast fix for chat gpt model:
        k = k.replace("gpt-3.5-turbo","gpt-3-5-turbo")
        dataset,split,metric,model,prompt = k.split(".")
        model = model.replace("gpt-3-5-turbo","gpt-3.5-turbo")
        instruction, cot_trigger, _ = prompt.split("_")
        # old: only cot_trigger
        # df.loc[dataset, (cot_trigger, model)] = v

        # new: instruction + cot_trigger
        col_name = instruction + "_" + cot_trigger + "_" + model
        if dataset not in data_dict:
            data_dict[dataset] = {}
        data_dict[dataset][col_name] = v

    # Create a dataframe from the dictionary
    df = pd.DataFrame.from_dict(data_dict, orient='index')

    # Calculate the average row and add it to the dataframe
    df.loc['Average'] = df.mean()

    # Round the dataframe to 2 decimal places
    df = df.round(2)

    return df

In [41]:
df = table_function_new(eval)
df

,None_None_command-xlarge-nightly,None_kojima-01_command-xlarge-nightly,None_zhou-01_command-xlarge-nightly,qa-10_None_command-xlarge-nightly,qa-12_None_command-xlarge-nightly,qa-13_None_command-xlarge-nightly,qa-16_None_command-xlarge-nightly,qa-17_None_command-xlarge-nightly,refl-01_None_command-xlarge-nightly,zhou-01-ins_None_command-xlarge-nightly,...,None_None_text-davinci-003,None_kojima-01_text-davinci-003,None_zhou-01_text-davinci-003,qa-10_None_text-davinci-003,qa-12_None_text-davinci-003,qa-13_None_text-davinci-003,qa-16_None_text-davinci-003,qa-17_None_text-davinci-003,refl-01_None_text-davinci-003,zhou-01-ins_None_text-davinci-003
commonsense_qa,0.52,0.48,0.73,0.70,0.70,0.76,0.67,0.61,0.70,0.73,...,0.73,0.67,0.67,0.70,0.79,0.67,0.58,0.64,0.76,0.73
med_qa,0.18,0.30,0.27,0.27,0.27,0.24,0.33,0.30,0.21,0.24,...,0.30,0.33,0.39,0.36,0.30,0.39,0.36,0.24,0.33,0.36
medmc_qa,0.27,0.15,0.42,0.27,0.30,0.36,0.36,0.36,0.24,0.33,...,0.36,0.36,0.48,0.45,0.36,0.36,0.39,0.42,0.39,0.33
open_book_qa,0.58,0.42,0.61,0.58,0.58,0.58,0.55,0.61,0.61,0.64,...,0.64,0.58,0.79,0.73,0.79,0.61,0.64,0.73,0.64,0.82
strategy_qa,0.55,0.58,0.24,0.12,0.09,0.12,0.33,0.30,0.06,0.09,...,0.61,0.64,0.64,0.58,0.67,0.70,0.55,0.58,0.64,0.64
worldtree,0.61,0.61,0.76,0.73,0.82,0.85,0.85,0.64,0.79,0.82,...,0.94,0.91,0.82,0.82,0.91,0.85,0.82,0.91,0.91,0.88
Average,0.45,0.42,0.51,0.44,0.46,0.48,0.52,0.47,0.43,0.47,...,0.60,0.58,0.63,0.61,0.64,0.60,0.56,0.59,0.61,0.63


In [49]:
df = (df[df.index=="Average"])
#df.to_csv("Average_values_df.csv")
df

,None_None_command-xlarge-nightly,None_kojima-01_command-xlarge-nightly,None_zhou-01_command-xlarge-nightly,qa-10_None_command-xlarge-nightly,qa-12_None_command-xlarge-nightly,qa-13_None_command-xlarge-nightly,qa-16_None_command-xlarge-nightly,qa-17_None_command-xlarge-nightly,refl-01_None_command-xlarge-nightly,zhou-01-ins_None_command-xlarge-nightly,...,None_None_text-davinci-003,None_kojima-01_text-davinci-003,None_zhou-01_text-davinci-003,qa-10_None_text-davinci-003,qa-12_None_text-davinci-003,qa-13_None_text-davinci-003,qa-16_None_text-davinci-003,qa-17_None_text-davinci-003,refl-01_None_text-davinci-003,zhou-01-ins_None_text-davinci-003
Average,0.45,0.42,0.51,0.44,0.46,0.48,0.52,0.47,0.43,0.47,...,0.6,0.58,0.63,0.61,0.64,0.6,0.56,0.59,0.61,0.63


In [12]:
import json, re
import pandas as pd
def clean_column(df, col_name):
    pattern = r"^.?'(.?)'.?'(.?)'.?'(.?)'.?'(.?)'.*$"
    return(df[col_name].apply(lambda x: re.sub(pattern, r"'\3", x)[1:]))

def json_to_dataframe_3(json_data):
    df_data = []
    for category, data in json_data.items():
        for subset, questions in data.items():
            for question in questions:
                for generated_cot in question['generated_cot']:
                    row = {
                        'dataset': category,
                        'split': subset,
                        'id': question['id'],
                        'model': generated_cot['model'],
                        'instruction': generated_cot['instruction'],
                        'cot_trigger': generated_cot["cot_trigger"],
                        'generated_cot': generated_cot['cot'],
                        'correct_answer': generated_cot['answers'][0]['correct_answer'],
                    }
                    df_data.append(row)
    df = pd.DataFrame(df_data)
    # df["model"] = clean_column(df, "model")
    return df


In [13]:
df["model"][0]

"'name': 'command-xlarge-nightly', 'temperature': 0, 'max_tokens': 512}"

In [10]:
path = "/home/kon/work/ThoughtSource/libs/cot/cot/datasets/thoughtsource/thoughtsource_33_paper.json"


with open(path, "r") as read_file:
    data = json.load(read_file)

df = json_to_dataframe_3(data)

df

,dataset,split,id,model,instruction,cot_trigger,generated_cot,correct_answer
0,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'command-xlarge-nightly', 'temperature...",None,None,The main purpose of farmers is to supply food.,True
1,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'text-davinci-003', 'temperature': 0, ...",None,None,E) supply food,True
2,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'flan-T5-xxl', 'temperature': 0, 'max_...",None,None,E,True
3,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'gpt-3.5-turbo', 'temperature': 0, 'ma...",None,None,E) supply food,True
4,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'text-davinci-002', 'temperature': 0, ...",None,None,\nE) supply food,True
...,...,...,...,...,...,...,...,...
11875,worldtree,test,worldtree_test_1659,"'name': 'flan-T5-xxl', 'temperature': 0, 'max_...",None,zhou-01,C is the dependent variable. The dependent var...,True
11876,worldtree,test,worldtree_test_1659,"'name': 'text-davinci-003', 'temperature': 0, ...",None,zhou-01,\n\nStep 1: The vertical axis on a graph is t...,False
11877,worldtree,test,worldtree_test_1659,"'name': 'text-davinci-002', 'temperature': 0, ...",None,zhou-01,\n\nThe first thing we need to do is identify...,True
11878,worldtree,test,worldtree_test_1659,"'name': 'command-xlarge-nightly', 'temperature...",None,zhou-01,\nThe y-axis is the vertical axis on a graph. ...,True
